In [ ]:
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator


sam_checkpoint = "/content/sam_vit_b_01ec64.pth"

sam = sam_model_registry["vit_b"](checkpoint=sam_checkpoint)
sam.to(device="cuda" if torch.cuda.is_available() else "cpu")

mask_generator = SamAutomaticMaskGenerator(sam)


def is_blood_region(image_rgb, mask, red_threshold=0.35):
    region = image_rgb[mask.astype(bool)]
    if region.size == 0:
        return False
    mean_color = np.mean(region, axis=0)  # [R,G,B]
    red_ratio = mean_color[0] / (mean_color.sum() + 1e-6)
    return red_ratio > red_threshold



image_path = "/content/photo_2025-08-29_02-29-21.jpg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


masks = mask_generator.generate(image_rgb)


blood_masks = []
for m in masks:
    mask = m["segmentation"]
    if is_blood_region(image_rgb, mask):
        blood_masks.append(mask)

print("✅ عدد الماسكات المحتملة للدم:", len(blood_masks))


output = image.copy()
for mask in blood_masks:
    blurred = cv2.GaussianBlur(image, (51, 51), 0)
    output[mask == 1] = blurred[mask == 1]


plt.figure(figsize=(12,6))

plt.subplot(1,2,1)
plt.title("Before (Original)")
plt.imshow(image_rgb)

plt.subplot(1,2,2)
plt.title("After (Blurred Regions)")
plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))

plt.show()
